In [ ]:
# Transformer 모델 구축 - Transformer 질의응답(QA) 모델
# 학습 목표 - 실무에서 사용되는 파이프라인 이해 및 적용


In [26]:
# QA Pre-trained 모델 테스트
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

base_model = 'monologg/koelectra-base-v3-finetuned-korquad' # 한국어 KorQuAD 데이터셋으로 파인 튜닝된 KoELECTRA QA 모델
tokenizer = AutoTokenizer.from_pretrained(base_model) # 토크나이저 로드
model = AutoModelForQuestionAnswering.from_pretrained(base_model) # QA 태스크용 헤드가 포함된 모델 로드

question = '서울은 어디에 있나요?'
context = '서울은 대한민국의 수도이며, 한반도의 중서부에 위치해 있습니다.'

# QA 모델 구조: BERT 기반 QS 모델은 입력을 [CLS]질문[SEP]문맥[SEP] 형태로 받는다
# 질문과 문맥을 하나의 입력으로 합쳐서 토큰화 결과를 모델에 전달해야 답변을 얻을 수 있는 구조
# [CLS] 서울 은 어디 에 있 나요 ? [SEP] 서울 은 대한민국 의 수도 이며 , 한반도 의 중서부 에 위치 해 있습니다 . [SEP]
inputs = tokenizer(question, context, return_tensors='pt') # (batch_size,seq_len) (1,seq_len)

# 모델에 입력을 전달하여 start_logits, end_logits 출력
# - start_logits: 답변 시작 위치에 대한 확률 분포
# - end_logits: 답변 끝 위치에 대한 확률 분포
outputs = model(**inputs)

# torch.argmax() 가장 확률이 높은 인덱스를 선택
# - answer_start: 답변 시작 토큰 위치, answer_end: 답변 끝 토큰 위치
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits) + 1 # 마지막 인덱스 포함

# 토큰을 문자열로 변환
# - tokenizer.convert_ids_to_tokens(): 토큰 ID->토큰 문자열
# - tokenizer.convert_tokens_to_string(): 토큰 문자열->사람이 읽을 수 있는 문장
answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end])
)

# Debug 확인
print('답변 시작 인덱스:', answer_start.item())
print('답변 끝 인덱스:', answer_end.item())

tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]) # 전체 토큰 확인
print('전체 토큰 시퀀스:', tokens)

answer_tokens = tokens[answer_start:answer_end] # 답변 토큰 범위 확인
print('답변 토큰:', answer_tokens)

answer = tokenizer.convert_tokens_to_string(answer_tokens) # 최종 답변
print('최종 답변:', answer)

답변 시작 인덱스: 18
답변 끝 인덱스: 22
전체 토큰 시퀀스: ['[CLS]', '서울', '##은', '어디', '##에', '있', '##나', '##요', '?', '[SEP]', '서울', '##은', '대한민국', '##의', '수도', '##이', '##며', ',', '한반도', '##의', '중서', '##부', '##에', '위치', '##해', '있', '##습', '##니다', '.', '[SEP]']
답변 토큰: ['한반도', '##의', '중서', '##부']
최종 답변: 한반도의 중서부


In [ ]:
# 데이터셋 로드: Hugging Face
from datasets import load_dataset

# 한국어 KorQuAD 데이터셋: train(60,407), validation(5,774)
korquad = load_dataset('squad_kor_v1')
print(korquad)

# 영어 SQuAD 2.0: train(130,319), validation(11,873)
squad = load_dataset('squad_v2')
print(squad)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})


README.md: 0.00B [00:00, ?B/s]

d:\AI\Pytorch\deep-learning\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\AI\.cache\huggingface\hub\datasets--squad_v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


squad_v2/train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

squad_v2/validation-00000-of-00001.parqu(…):   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})
